# A basic example of reduced order quadrature

In [Antil _et al_ (2013)](https://arxiv.org/abs/1210.0577) the concept of _reduced order quadrature_ (ROQ) was introduced. The method has, in particular, found extensive use in the field of gravitational wave astronomy, following the initial work of [Canizares _et al_, 2013](https://ui.adsabs.harvard.edu/abs/2013PhRvD..87l4005C/abstract), where it is used to **massively speed-up the computation of likelihoods** during stochastic sampling (via, e.g., [MCMC](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo) or [nested sampling](https://en.wikipedia.org/wiki/Nested_sampling_algorithm)). In this note I will attempt to provide a brief, simple example of the "quadrature" part of this method and its use. I will not go into detail about the slightly more complex world of creating a "[reduced order model](https://en.wikipedia.org/wiki/Model_order_reduction)".

## What's a "quadrature"?

Within reduced order quadrature, the "[quadrature](https://en.wikipedia.org/wiki/Quadrature_(mathematics))" term is just another way of saying numerical integration, i.e., calculating an integral by approximating it as a sum. E.g.,: 

$$
\int_a^b f(x) {\rm d}x \approx \sum_{i=1}^N f(x_i) \Delta x,
$$

where $f(x)$ is the integrand (i.e., the function to be integrated) between the limits $a$ and $b$, $N$ is the number of intervals over which to split the sum and therefore $x_1 = a$ and $\Delta x = (b - a) / N$. More accurate numerical approximations such as the [trapezium rule](https://en.wikipedia.org/wiki/Trapezoidal_rule) can be used.

In the papers above, and what we will assume in this example, is that the integrand is a very specific function: the natural logarithm of a [Gaussian likelihood](https://en.wikipedia.org/wiki/Normal_distribution) function:

$$
\ln{L(\vec{\theta})} = -\int_{-\infty}^{\infty} \frac{\left(d(t) - m(t;\vec{\theta})\right)^2}{2\sigma(t)^2} {\rm d}t,
$$

where $d(t)$ is some data as a function of $t$ (which might be, for example, _time_), $m(t;\vec{\theta})$ is a predictive model of some process as a function of $t$ and a set of parameters $\vec{\theta}$, and $\sigma(t)$ is an estimate of the noise standard deviation of the data as a function of $t$. In many statistical application, this is the form of the likelihood function you might want to use to, e.g., maximise and find the "best fit" model parameters $\vec{\theta}$, or sample from during the process of approximating a posterior probability distribution on the model parameters.

In realistic applications, the data $d$ is discrete rather than continuous, so the integral naturally becomes a sum with finite bounds (e.g., the start and end times of the data, or a lower and upper frequency range):

$$
\ln{L(\vec{\theta})} \propto \sum_{i=1}^{N} \frac{\left(d_i - m(t_i;\vec{\theta})\right)^2}{\sigma_i^2},
$$

where $N$ is the number of data points.

Reduced order quadrature is a way of speeding up the calculation of this sum by reducing the number in the summation from $N$ to $M$, where $M \ll N$. This is particularly useful if the model $m$ is computationally expensive, i.e., it takes a long time to evaluate it for large numbers of data points, and/or $N$ is intrinsically large.

To get started on this path we will expand and separate out the terms in the above equation:

\begin{equation}
\ln{L(\vec{\theta})} \propto \left(\sum_{i=1}^{N} \frac{d_i^2}{\sigma_i^2}\right) + \left(\sum_{i=1}^{N} \frac{m_i(\vec{\theta})^2}{\sigma_i^2}\right) - \left(2 \sum_{i=1}^{N} \frac{d_i m_i(\vec{\theta})}{\sigma_i^2}\right),\
\label{eq:expanded}
\tag{1}
\end{equation}

where for compactness I've switched $m(t_i;\vec{\theta})$ to $m_i(\vec{\theta})$. In the subsequenct equations I will switch $d_i/\sigma_i \rightarrow d_i$ and $m_i / \sigma_i \rightarrow m_i$, i.e., the data and model terms will be assumed to be weighted by the noise. The first term on the right hand side of the equation does not depend on the parameters $\vec{\theta}$, so no matter what $\vec{\theta}$ is used for the likelihood function it will always be the same, i.e., it ($\mathcal{K} = \sum_{i=1}^N d_i^2$) only needs to be calculated once. The other terms must be calculated each time $\vec{\theta}$ changes. In some cases, e.g., that described in [Canizares _et al_, 2013](https://ui.adsabs.harvard.edu/abs/2013PhRvD..87l4005C/abstract), the second term involving the square of the model is analytic and therefore quick to calculate, but I'll not assume that here.



## How do you reduce the number of points?

Reducing the number of points over which you can sum relies on the model $m$ being reducible into a linear superposition of a relatively small number of (scaled) basis vectors. This is known as [reduced order modelling](https://en.wikipedia.org/wiki/Model_order_reduction). Most generally the set of basic vectors must be "learned" by training in some way using models evaluated over the parameter space $\vec{\theta}$. After training, there should be a set of basic vectors $b$ such that:

\begin{equation}
m(t; \vec{\theta}) \approx \sum_{j=1}^M C_j(\vec{\theta}) b_j(t)
\label{eq:basis}
\tag{2}
\end{equation}

where it is the coefficients $C$ that depend on the parameters $\vec{\theta}$, but not the basic vectors.

If we return to the third term in Equation $\eqref{eq:expanded}$ and substitute in Equation $\eqref{eq:basis}$ we get:

\begin{equation}
\sum_{i=1}^N d_i m_i(\vec{\theta}) \approx \sum_{i=1}^N d_i \left(  \sum_{j=1}^M C_j(\vec{\theta}) b_{ji} \right) \equiv \sum_{j=1}^M C_j(\vec{\theta}) \left(\sum_{i=1}^N d_i  b_{ji}\right),
\label{eq:rhs3}
\tag{3}
\end{equation}

where $b_{ji} = b_j(t_i)$.

> Note: we can see that the rearrangement of the summation terms works by expanding things for $N = M = 2$:
> $d_1 \left(C_1b_{11} + C_2b_{21}\right) + d_2\left(C_1b_{12} + C_2b_{22}\right) \equiv C_1\left(d_1b_{11} + d_2b_{12}\right) + C_2\left(d_1b_{21} + d_2b_{22} \right)$.

Neither the basis vectors $b$ nor the data depend on the parameters $\vec{\theta}$, so the sum over $N$ for each basis vector

$$
D_j = \sum_{i=1}^N d_i  b_{ji}
$$

can all be pre-calculated. This means that Equation $\eqref{eq:rhs3}$ becomes a sum over $M$:

$$
\sum_{i=1}^N d_i m_i(\vec{\theta}) \approx \sum_{j=1}^M C_j(\vec{\theta}) D_j.
$$

Something similar can be done for the second term in Equation $\eqref{eq:expanded}$. First we have:

$$
\sum_{i=1}^N m_i(\vec{\theta})^2 \approx \sum_{i=1}^N \sum_{j=1}^M C_j(\vec{\theta})^2 b_{ji}^2 \equiv \sum_{j=1}^M C_j(\vec{\theta}) \sum_{i=1}^N b_ji^2.
$$

Again, the sum over $N$ for the square of the basis vectors does not involve $\vec{\theta}$, so these can be pre-computed:

$$
B_j = \sum_{i=1}^N b_{ji}^2,
$$

and we get another sum just over $M$:

$$
\sum_{i=1}^N m_i(\vec{\theta})^2 \approx \sum_{j=1}^M C_j(\vec{\theta}) B_j.
$$

Finally, Equation $\eqref{eq:expanded}$ can become:

\begin{equation}
\ln{L(\vec{\theta})} \propto \mathcal{K} + \sum_{j=1}^M C_j(\vec{\theta}) B_j - 2\sum_{j=1}^M C_j(\vec{\theta}) D_j.
\label{eq:roqlike}
\tag{4}
\end{equation}

So, provided $M \ll N$, there should be an approximately $N/M$ speed-up in calculating Equation $\eqref{eq:roqlike}$ compared to $\eqref{eq:expanded}$.

## How do you calculate the $C$ coefficients?

To calculate the $C$ coefficients required for Equation $\eqref{eq:roqlike}$ you need to apply a bit of linear algebra. For a particular set of parameters $\vec{\theta}_k$ you need to evaluate the model $m(\vec{\theta}_k)$ at $M$ points (optimally chosing which points is beyond the scope of this note). Say we had only 2 basis vectors, i.e., $M=2$, then we can define the two simultaneous equations:

$$
\begin{eqnarray}
m(t_l;\vec{\theta}_k) & = & \sum_{j=1}^2 C_j(\vec{\theta}_k) b_{jl} \equiv C_1(\vec{\theta}_k) b_{1l} + C_2(\vec{\theta}_k) b_{2l} \\
m(t_n;\vec{\theta}_k) & = & \sum_{j=1}^2 C_j(\vec{\theta}_k) b_{jn} \equiv C_1(\vec{\theta}_k) b_{1n} + C_2(\vec{\theta}_k) b_{2n},
\end{eqnarray}
$$

so that we have (in matrix notation):

$$
\left( \begin{array}{c} m_l(\vec{\theta}_k) \\ m_n(\vec{\theta}_k) \end{array} \right) = \left( \begin{array}{c} C_1(\vec{\theta}_k) \\ C_2(\vec{\theta}_k) \end{array} \right) \left( \begin{array}{cc} b_{1l} & b_{2l} \\ b_{1n} & b_{2n} \end{array} \right) 
$$

then the values of $m_l(\vec{\theta}_k)$ and $m_n(\vec{\theta}_k)$ can be explicitly calculated from the model at the points $l$ and $n$. The basis vector matrix:

$$
\mathbf{B} = \left( \begin{array}{cc} b_{1l} & b_{2l} \\ b_{1n} & b_{2n} \end{array} \right),
$$

can then be inverted to solve the equations for $C_1(\vec{\theta}_k)$ and $C_2(\vec{\theta}_k)$.

It could be that the model is intrinsically the sum of a set of functions $f$:
$$
m(t; \vec{\theta}) = \sum_{i=1}^M C_i f_i(t; \vec{\theta}_i),
$$

where $C$ are scale factors for each of the functions and $\vec{\theta}_i$ are the subset of model parameters



A simple example is a model that is explicitly defined to be the sum of two simple functions, e.g.:

$$
f(t; A, B) = A g(t) + B h(t),
$$

with scaling factors of $A$ and $B$ for each. We could for example set $g(t)$ to be the sine function with a fixed angular frequency:

$$
g(t) = sin(2.3 t),
$$

and $h(t)$ to be the function for a straight line with a fixed gradient and $y$-intercept:

$$
h(t) = 3.4t - 1.2.
$$


In [1]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt